__Optimization Modeling using `PuLP`__

https://medium.com/analytics-vidhya/optimization-modelling-in-python-scipy-pulp-and-pyomo-d392376109f4

In [1]:
import pandas as pd

from pulp import LpProblem, LpVariable, LpStatus, lpSum, value
from pulp import LpMaximize, LpMinimize

In [2]:
customer_demand = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}

factory_capacity = {1: 500, 2: 500, 3: 500}

customers = customer_demand.keys()

factories = factory_capacity.keys()

transportation_cost = {
    (1, 1): 4,
    (1, 2): 6,
    (1, 3): 9,
    (2, 1): 5,
    (2, 2): 4,
    (2, 3): 7,
    (3, 1): 6,
    (3, 2): 3,
    (3, 3): 3,
    (4, 1): 8,
    (4, 2): 5,
    (4, 3): 3,
    (5, 1): 10,
    (5, 2): 8,
    (5, 3): 4,
}

In [3]:
goods = LpVariable.dicts(
    "amount of goods",
    ((customer, factory) for customer in customers for factory in factories),
    lowBound=0,
    cat="Integer",
)

factory_goods = LpVariable.dicts(
    "factory goods", (factory for factory in factories), lowBound=0, cat="Integer"
)

In [4]:
model = LpProblem("Transportation cost minimization", LpMinimize)

# Objective function
model += lpSum(
    [
        goods[customer, factory] * transportation_cost[customer, factory]
        for customer, factory in goods
    ]
)

/usr/local/lib/python3.7/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [5]:
# Constraint: sum of goods == customer demand
for customer in customers:
    model += (
        lpSum(
            [
                goods[customer, factory]
                for factory in factories
                if (customer, factory) in goods
            ]
        )
        == customer_demand[customer]
    )

# Constraint: sum of goods <= factory capacity
for factory in factories:
    for customer in customers:
        factory_goods[factory] += lpSum([goods[customer, factory]])
    model += factory_goods[factory] <= factory_capacity[factory]

In [6]:
model.solve()

LpStatus[model.status]

'Optimal'

In [7]:
value(model.objective)

3350.0

In [8]:
output = []

for (customer, factory) in goods:
    customer_amount = goods[customer, factory].varValue
    if customer_amount > 0:
        output.append([customer, factory, customer_amount])

output = pd.DataFrame(output, columns=["customer", "factory", "customer_amount"])
output.sort_values(["customer", "factory", "customer_amount"], ascending=True)

,customer,factory,customer_amount
0,1,1,80.0
1,2,2,270.0
2,3,2,230.0
3,3,3,20.0
4,4,3,160.0
5,5,3,180.0
